In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [3]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_dir='data_summary').load_data()
single_document = SimpleDirectoryReader(input_files=['data/case1.txt']).load_data()

In [4]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=8192)
nodes = splitter.get_nodes_from_documents(documents)
nodes_single = splitter.get_nodes_from_documents(single_document)

In [5]:
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = Gemini(api_key=GEMINI_API_KEY)
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

/Users/lakshya/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lakshya/miniforge3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/lakshya/miniforge3/lib/python3.9/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [6]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes_single)
vector_index = VectorStoreIndex(nodes)

In [7]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [8]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to legal Case Files"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the legal cases provided."
    ),
)

In [10]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [11]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The question asks for a summary of the document, which aligns with the purpose of summarization..
The document is a Supreme Court of India judgment regarding a dispute between two companies, M/s. N.N. Global Mercantile Pvt. Ltd. and M/s. Indo Unique Flame Ltd. The case revolves around the enforceability of an arbitration agreement embedded in an unstamped work order. The court examines the doctrine of separability of arbitration agreements and the implications of non-payment of stamp duty on the work order. The judgment also addresses the arbitrability of allegations of fraudulent invocation of a bank guarantee. Ultimately, the court sets aside the High Court's judgment and directs the parties to proceed with arbitration, while also referring the issue of the enforceability of an arbitration agreement in an unstamped contract to a Constitution Bench for authoritative determination. 



In [12]:
print(len(response.source_nodes))

3


In [13]:
response = query_engine.query(
    "A construction company entered into a contract with a government agency for the construction of a bridge. The contract specified that the arbitration proceedings would be held in Mumbai. However, the government agency filed an application under Section 34 of the Arbitration and Conciliation Act, 1996 before the Delhi High Court challenging the arbitral award. The construction company argues that the Delhi High Court lacks jurisdiction as the seat of arbitration was designated as Mumbai. Determine the relevant legal principles and jurisdictional considerations in this case."
)
print(str(response))

Selecting query engine 1: The question asks to determine legal principles and jurisdictional considerations, which requires retrieving specific context from the legal case provided..
The relevant legal principles in this case involve the distinction between the "seat" and "venue" of arbitration, as well as the concept of exclusive jurisdiction. The "seat" of arbitration determines the applicable law and the courts with supervisory jurisdiction, while the "venue" refers to the physical location of hearings. 

The construction company's argument that the Delhi High Court lacks jurisdiction is based on the principle that the court at the "seat" of arbitration has exclusive jurisdiction over disputes arising from the arbitration proceedings. Since the contract specified Mumbai as the seat of arbitration, the construction company contends that the Delhi High Court is not the appropriate forum for the government agency's challenge to the arbitral award.

The government agency, on the other h